<a href="https://colab.research.google.com/github/nbisso/TP2-Orga-7506/blob/main/TP_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://realpython.com/train-test-split-python-data/

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import f1_score

In [ ]:
df_train_labels = pd.read_csv('/content/drive/MyDrive/datasets_tp2/train_labels.csv', index_col = 'building_id')

In [ ]:
df_train_values = pd.read_csv('/content/drive/MyDrive/datasets_tp2/train_values.csv', 
                              dtype = {'land_surface_condition':'category', 'foundation_type':'category'}, index_col = 'building_id')

In [ ]:
df_train_values.dtypes

geo_level_1_id                               int64
geo_level_2_id                               int64
geo_level_3_id                               int64
count_floors_pre_eq                          int64
age                                          int64
area_percentage                              int64
height_percentage                            int64
land_surface_condition                    category
foundation_type                           category
roof_type                                   object
ground_floor_type                           object
other_floor_type                            object
position                                    object
plan_configuration                          object
has_superstructure_adobe_mud                 int64
has_superstructure_mud_mortar_stone          int64
has_superstructure_stone_flag                int64
has_superstructure_cement_mortar_stone       int64
has_superstructure_mud_mortar_brick          int64
has_superstructure_cement_morta

In [ ]:
df_train_labels.head()

,damage_grade
building_id,
802906,3
28830,2
94947,3
590882,2
201944,3


In [ ]:
df_train_values.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,1,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,f,x,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,f,x,s,d,0,1,0,0,0,0,1,1,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,f,x,s,d,1,0,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#df_train_values = df_train_values.set_index('building_id')

In [ ]:
columnas = ['foundation_type', 
                     'area_percentage', 
                     'height_percentage',
                     'count_floors_pre_eq',
                     'land_surface_condition',
                     'has_superstructure_cement_mortar_stone', 'damage_grade']

In [ ]:
df_train_values['damage_grade'] = df_train_labels['damage_grade']


In [ ]:
df = df_train_values[columnas]
df['damage_grade'] = df['damage_grade'] -1
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,foundation_type,area_percentage,height_percentage,count_floors_pre_eq,land_surface_condition,has_superstructure_cement_mortar_stone,damage_grade
building_id,,,,,,,
802906,r,6,5,2,t,0,2
28830,r,8,7,2,o,0,1
94947,r,5,5,2,t,0,2
590882,r,6,5,2,t,0,1
201944,r,8,9,3,t,0,2


In [ ]:
X,y = df.iloc[:, :-1], df.iloc[:, -1]
X

,foundation_type,area_percentage,height_percentage,count_floors_pre_eq,land_surface_condition,has_superstructure_cement_mortar_stone
building_id,,,,,,
802906,r,6,5,2,t,0
28830,r,8,7,2,o,0
94947,r,5,5,2,t,0
590882,r,6,5,2,t,0
201944,r,8,9,3,t,0
...,...,...,...,...,...,...
688636,r,6,3,1,n,0
669485,r,6,5,2,t,0
602512,r,6,7,3,t,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.333337170617)


In [ ]:
y_test

building_id
142408    2
732642    1
187301    1
773333    1
679639    2
         ..
718444    1
28212     1
978655    1
31091     1
123915    2
Name: damage_grade, Length: 86868, dtype: int64

In [ ]:
lgb_train = lgb.Dataset(X_train, label = y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference= lgb_train)

In [ ]:
params = {
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_classes': 3,
    'num_leaves': 40,
    'learning_rate': 0.05,
    'max_depth': 8
}

In [ ]:
#https://neptune.ai/blog/lightgbm-parameters-guide
model = lgb.train(params,
                lgb_train,
                num_boost_round=200,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)

In [ ]:
pred = model.predict(X_test)

In [ ]:
labels_pred = [np.argmax(v) + 1 for v in pred]
len(labels_pred)

86868

In [ ]:
error = f1_score(y_test, labels_pred, average ='micro')
error

0.3127734033245844

In [ ]:
df_result = pd.DataFrame(data = {'building_id': [], 'damage_grade': []})
df_result['building_id'] = X_test.reset_index()['building_id']
df_result['damage_grade'] = labels_pred
df_result

,building_id,damage_grade
0,142408,2
1,732642,2
2,187301,2
3,773333,1
4,679639,2
...,...,...
86863,718444,2
86864,28212,2
86865,978655,2
86866,31091,2


In [ ]:
df_final = df.reset_index().merge(df_result, how='inner', on= 'building_id').head()

df_final

,building_id,foundation_type,area_percentage,height_percentage,count_floors_pre_eq,land_surface_condition,has_superstructure_cement_mortar_stone,damage_grade_x,damage_grade_y
0,201944,r,8,9,3,t,0,2,2
1,441126,r,8,6,2,t,0,1,2
2,989500,i,13,4,1,t,0,0,1
3,452227,u,8,4,1,t,0,1,2
4,563431,r,4,4,2,n,0,1,2


In [ ]:
df_final.to_csv('/content/drive/MyDrive/datasets_tp2/12072021-training-result.csv',\
index= False, header=True,sep=',')